<a id='intro'></a>

## Содержание


[Введение](#chapter0)

[Загрузка данных](#chapter1)

[Умножение матриц](#chapter2)

[Алгоритм преобразования](#chapter3)

[Проверка алгоритма](#chapter4)

[Общий вывод](#chapter5)


<a id='chapter0'></a>

# Введение

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

<a id='chapter1'></a>

# Загрузка данных

In [1]:
#загрузим необходимые библиотеки
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

In [2]:
#сохраним данные в data
data = pd.read_csv('/datasets/insurance.csv')


Посмотрим информацию о загруженных данных.


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0



В нашей таблице данных пять столбцов:

* Пол - пол застрахованного

* Возраст - возраст застрахованного

* Зарплата - зарплата застрахованного

* Члены семьи - количество членов его семьи

* Страховые выплаты - количество страховых выплат клиенту за последние 5 лет



Проведем предобработку данных: проверим есть ли дублирующиеся строки и пропуски.


In [5]:
#проверка на наличие пропусков
data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [6]:
#проверка количества дубликатов
data.duplicated().sum()

153


Пропусков нет, но есть дублирующиеся строки. Их немного, можем их удалить.


In [7]:
#удалим дубликаты
data = data.drop_duplicates()
data = data.dropna().reset_index(drop=True)


Теперь все в порядке: нет ни пропусков, ни дубликатов. Можем разделить нашу таблицу данных на две: матрицу признаков и вектор целевого признака.

В качестве целевого признака у нас столбец с количеством страховых выплат клиенту.


In [8]:
#выделяем обучающие признаки
features = data.drop('Страховые выплаты', axis=1).values
#выделяем целевой признак
target = data['Страховые выплаты'].values


Теперь, когда у нас есть матрица признаков и вектор целевого признака, можно приступить к следующему этапу - продумать способ защиты персональной информации.


[К оглавлению](#intro)


<a id='chapter2'></a>

# Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.


Нам необходимо продумать алгоритм шифрования исходной информации, чтобы защитить персональные данные. Если умножить матрицу признаков на случайную матрицу, то восстановить информацию, не зная ее, будет невозможно. Однако нужно определить, не изменится ли при этом качество линейной регрессии?


Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Обоснование:**



Преобразуем формулу обучения, вместо Х подставим ХР:

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$


$$
w' = (P^T X^T XP)^{-1} P^T X^T y
$$

Раскроем скобки - возведем в степень -1:

$$
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$


Нам известно, что

$$
X^{-1} X = I
$$


Значит мы можем преобразовать полученную формулу следующим образом:

$$
w' = P^{-1} (X^T X)^{-1} X^T y
$$


Видим, что

$$
w' = P^{-1} w
$$


Теперь посмотрим, как изменятся предсказания при умножении матрицы Х на матрицу Р:

$$
a' = XPw'
$$

$$
a' = XP P^{-1}w
$$


То есть


$$
a' = Xw = a
$$

**Ответ:** Качество линейной регрессии не изменится.


[К оглавлению](#intro)


<a id='chapter3'></a>

# Алгоритм преобразования


Итак, на предыдущем этапе мы доказали, что качество линейной регрессии не изменится, если матрицу признаков умножить на некую случайную матрицу. Однако, эта матрица должна обладать некоторыми свойствами, а именно:

* матрица P должна быть обратима;

* если у матрицы Х размерность (n, m), то у матрицы Р должна быть размерность (m, m).



Создадим такую матрицу.


In [9]:
m = features.shape[1]
#создаем случайную матрицу
P = np.random.normal(size=(m, m))


Проверим, является ли матрица обратимой. Квадратная матрица обратима тогда и только тогда, когда она невырождена, т.е. ее определитель не равен нулю.

In [10]:
print(np.linalg.det(P))

-0.06108827631612173



Определитель не равен нулю, значит можем продолжать работу с полученной матрицей.


[К оглавлению](#intro)


<a id='chapter4'></a>

# Проверка алгоритма


Напишем код для обучения и проверки модели методом линейной регрессии.


In [11]:
#модель линейной регрессии
class LinearRegression:
    #объявляем функцию обучения модели
    def fit(self, train_features, train_target):
        #матрица признаков с дополнительным столбцом из единиц
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        #вектор целевого признака
        y = train_target
        #вектор параметров модели
        w = np.linalg.inv(X.T @ X) @ X.T @ y
        self.w = w[1:]
        self.w0 = w[0]
    #объявляем функцию предсказания
    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [12]:
#обучим модель на первоначальных данных
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
#найдем метрику R2
print(r2_score(target, predictions))

0.4302010044852068


In [13]:
#обучим модель на зашифрованных данных
model_new = LinearRegression()
model_new.fit(features @ P, target)
predictions_new = model_new.predict(features @ P)
#найдем метрику R2
print(r2_score(target, predictions_new))

0.43020095128027913



Видим, что значение метрики R2 практически идентично в обоих случаях. Таким образом, мы убедились, что выбранный алгоритм шифрования не меняет качество линейной регрессии.


[К оглавлению](#intro)


<a id='chapter5'></a>

# Вывод



В данной работе мы разработали действенный метод шифрования персональных данных клиентов для страховой компании "Хоть потоп". Корректность его работы была доказана не только математически, но и подтверждена экспериментально.

Мы запрограммировали алгоритм защиты данных и написали модель линейной регрессии, проверив наши данные на ее работе.


[К оглавлению](#intro)
